**Accessing via Huggingface API**

In [ ]:
pip install huggingface_hub

In [ ]:
pip install pandas requests

In [ ]:
import pandas as pd
import requests
import time

# -------------------------------------------------
# CONFIGURATION
# -------------------------------------------------
HF_API_KEY = "api_XXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX"
MODEL_ID = "NousResearch/Hermes-3-Llama-3.1-8B"

# CSV splits to process
csv_files = ["input.csv"]  # change according to the dataset

# change according to the dataset
DATASET_DESCRIPTION = "VAST isimli, 2020 yılında oluşturulmuş bu dataset, The New York Times’in ‘Room for Debate’ bölümündeki yorumların etiketlenmesiyle hazırlanmış olup, özellikle tutum (stance) tespiti görevleri için tasarlanmıştır. Politika (örneğin, 'Filistin devleti'), eğitim (örneğin, 'imtiyazlı okullar') ve halk sağlığı (örneğin, 'çocukluk aşıları') gibi geniş temaları kapsayan çeşitli konuları kapsar. Ayrıca, ‘kampüste silahlar’ ile ‘kampüste ateşli silahlar’ gibi benzer ifadelerin bir arada bulunduğu örnekleri de içerir."

# -------------------------------------------------
# HELPER: Generate 'Context' text from model
# -------------------------------------------------
def generate_context(text, target):
    """
    Calls the Hugging Face Inference API with the prompt
    and returns the model's response as a string.
    """
    # Build prompt
    prompt = f"""Sosyal bir uzman olduğunu varsayarak, aşağıda {DATASET_DESCRIPTION} datasetinin kısa bir pasajı verilmiştir, lütfen adım adım düşün, metindeki anahtar kelimeleri çıkar, yazarın ima ettiği duyguları, retorik araçları vb. analiz et, son olarak yazarın Target hakkındaki duruşunu kısaca analiz et, sonuca varmadan analiz sürecini vermeye dikkat et. Ayrıca açıklama 100 kelimeyi geçmesin ve hiçbir şekilde link içermesin.
Passage: {text}
Target: {target}
"""

    # Prepare JSON payload for the HF Inference API
    payload = {
        "inputs": prompt,
        "parameters": {
            "max_new_tokens": 500,
        }
    }

    # Make request
    headers = {
        "Authorization": f"Bearer {HF_API_KEY}",
        "Content-Type": "application/json"
    }
    HF_API_URL = f"https://api-inference.huggingface.co/models/{MODEL_ID}"

    response = requests.post(HF_API_URL, headers=headers, json=payload)

    # If no success, handle errors
    if response.status_code != 200:
        print(f"Error: {response.status_code}, {response.text}")
        return ""

    data = response.json()
    # data is typically [{"generated_text": "..."}]
    if isinstance(data, list) and len(data) > 0 and "generated_text" in data[0]:
        return data[0]["generated_text"]
    elif isinstance(data, dict) and "generated_text" in data:
        return data["generated_text"]

    return ""

# -------------------------------------------------
# MAIN: For each CSV, add a "Context" column
# -------------------------------------------------
for csv_file in csv_files:
    print(f"Processing: {csv_file}")
    df = pd.read_csv(csv_file, encoding="utf-8")

    # create a new column "Context" by calling generate_context row by row.
    context_list = []
    for i, row in df.iterrows():
        target_val = row["Target"]
        text_val = row["Text"]

        # Generate context from LLM
        context_text = generate_context(text_val, target_val)


        context_list.append(context_text)

    # Add the new column
    df["Context"] = context_list

    # Save the updated CSV
    output_name = csv_file.replace(".csv", "_with_context.csv")
    df.to_csv(output_name, index=False, encoding="utf-8-sig")
    print(f"Saved: {output_name}")


**Accessing Locally**

In [ ]:
pip install torch transformers accelerate sentencepiece

In [ ]:
!git lfs install
!git clone https://huggingface.co/NousResearch/Hermes-3-Llama-3.1-8B

In [ ]:
pip install pandas

In [ ]:
import os
import torch
import pandas as pd
from transformers import AutoTokenizer, AutoModelForCausalLM

############################
# CONFIGURATION
############################
MODEL_PATH = "model_path"  # local folder with the model
INPUT_CSV = "input_csv_path"  # input CSV with 'Text' and 'Target' columns         
OUTPUT_CSV = "output_csv_path"     # output CSV with new 'Context' column

# change according to the dataset
DATASET_DESCRIPTION = (
    "VAST isimli, 2020 yılında oluşturulmuş bu dataset, The New York Times’in ‘Room for Debate’ bölümündeki yorumların "
    "etiketlenmesiyle hazırlanmış olup, özellikle tutum (stance) tespiti görevleri için tasarlanmıştır. Politika (örneğin, 'Filistin devleti'),"
    "eğitim (örneğin, 'imtiyazlı okullar') ve halk sağlığı (örneğin, 'çocukluk aşıları') gibi geniş temaları kapsayan çeşitli konuları kapsar. "
    "Ayrıca, ‘kampüste silahlar’ ile ‘kampüste ateşli silahlar’ gibi benzer ifadelerin bir arada bulunduğu örnekleri de içerir."
)

############################
# LOAD MODEL LOCALLY
############################
print("Loading tokenizer...")
tokenizer = AutoTokenizer.from_pretrained(MODEL_PATH)
print("Loading model...")
model = AutoModelForCausalLM.from_pretrained(
    MODEL_PATH,
    torch_dtype=torch.float16,  # or bfloat16, etc. adjust as needed
    device_map="auto",         # automatically map to GPU or CPU
)

############################
# GENERATION FUNCTION
############################
def generate_context(text, target):
    """
    Generates context from the local LLaMA model:
    - No external links
    - Under ~100 words
    """
    # Prompt: no links, under 100 words
    prompt = f"""
Sosyal bir uzman olduğunu varsayarak, aşağıda {DATASET_DESCRIPTION} datasetinden
kısa bir pasaj verilmiştir. Lütfen adım adım düşün, metindeki anahtar kelimeleri çıkar;
yazarın ima ettiği duyguları, retorik araçları vb. analiz et;
ve son olarak yazarın Target hakkındaki duruşuna dair kısa bir analiz yap, sonuca varmadan analiz sürecini vermeye dikkat et.
Ayrıca açıklama 100 kelimeyi geçmesin ve hiçbir şekilde link içermesin.

Passage: {text}
Target: {target}
"""

    # Encode prompt
    inputs = tokenizer(prompt, return_tensors="pt").to(model.device)

    # Generate text
    with torch.no_grad():
        output_ids = model.generate(
            **inputs,
            max_new_tokens=512,        # limit generation
            do_sample=True
        )

    # Decode the entire output
    full_output = tokenizer.decode(output_ids[0], skip_special_tokens=True)

    # Separate out the part after the prompt
    prompt_len = len(tokenizer.encode(prompt))
    response = tokenizer.decode(output_ids[0][prompt_len:], skip_special_tokens=True)

    # Optional: a quick post-process to ensure we don't exceed ~100 words
    #   (model might still exceed if it doesn't follow instructions)
    response_words = response.strip().split()
    if len(response_words) > 100:
        response = " ".join(response_words[:100])

    return response.strip()

############################
# MAIN FLOW
############################
def main():
    print(f"Reading input CSV: {INPUT_CSV}")
    df = pd.read_csv(INPUT_CSV, encoding="utf-8")

    if "Text" not in df.columns or "Target" not in df.columns:
        raise ValueError("Input CSV must have 'Text' and 'Target' columns.")

    # Generate 'Context' for each row
    contexts = []
    for i, row in df.iterrows():
        text_val = str(row["Text"])
        target_val = str(row["Target"])
        ctx = generate_context(text_val, target_val)
        contexts.append(ctx)
        if (i+1) % 10 == 0:
            print(f"Processed {i+1} rows...")

    df["Context"] = contexts

    # Save the output
    print(f"Saving to {OUTPUT_CSV}")
    df.to_csv(OUTPUT_CSV, index=False, encoding="utf-8-sig")
    print("Done!")

if __name__ == "__main__":
    main()
